In [9]:
import pandas as pd
import numpy as np

import scipy.io as sc
import scipy.stats
from scipy.fft import fft, fftfreq
from scipy.fftpack import fft, ifft 
from os import listdir
import os.path
import shutil 
from IPython.display import clear_output

import matplotlib.pyplot as plt
import seaborn as sns

# Creating folder called Corrected and deleting previous folder

In [10]:
# success = True
# while success:
#     try:
#         shutil.rmtree('corrected')
#         success = False
#     except:
#         success = False

In [11]:
# success = True
# while success:
#     try: 
#         list_of_folders = listdir("cutting_tests_processed")
#         for folder in list_of_folders:
#             os.makedirs(f'corrected/{folder}')
#         success = False
#     except:
#         success = False

# Manually classifing data for the presence of chatter into new folder called corrected

In [12]:
### Print graphs to use as references

# path = 'split_cutting_tests_processed/2inch_stickout'

# sem_chatter = [j for j in listdir(path) if j[0] == 's' and
#                                         '_noise_added' not in j]

# com_chatter = [j for j in listdir(path) if j[0] == 'c' and
#                                         '_noise_added' not in j]



# for j in range(5):
#     plot = pd.read_csv(f'{path}/{sem_chatter[j]}')
#     fig, ax = plt.subplots(1,2, figsize = (10,4))

#     ax[0].plot(plot['t'], plot['y'])
#     fft = np.fft.fft(plot['y'])
#     fftfreq = np.fft.fftfreq(len(plot['t']), plot['t'][1] - plot['t'][0])
#     ax[1].plot(fftfreq[10:int(len(fft)/2)], np.abs(fft[10:int(len(fft)/2)])/len(fft), c = 'k')
#     fig.suptitle(sem_chatter[j])


#     plt.show()

# for j in range(5):
#     plot = pd.read_csv(f'{path}/{com_chatter[j]}')
#     fig, ax = plt.subplots(1,2, figsize = (10,4))

#     ax[0].plot(plot['t'], plot['y'])
#     fft = np.fft.fft(plot['y'])
#     fftfreq = np.fft.fftfreq(len(plot['t']), plot['t'][1] - plot['t'][0])
#     ax[1].plot(fftfreq[10:int(len(fft)/2)], np.abs(fft[10:int(len(fft)/2)])/len(fft), c = 'k')
#     fig.suptitle(com_chatter[j])


#     plt.show()
    

In [13]:
# def main_():
#     '''
#     changes the split files first letter to match it's correct classification to
#     the presence of chatter according to the users input and saves the file with
#     the corect name to the folder "corrected" with the same folders as 
#     split_cutting_tests_processed.
#     '''
#     folders = listdir('split_cutting_tests_processed')
#     print(folders)
#     # folder = ''
#     path_split = 'split_cutting_tests_processed'
#     path_corrected = 'corrected'

    
#     for folder in folders:
#         list_of_files = listdir(f'{path_split}/{folder}') 
#         for file in list_of_files:
#             if file[0] != 's' and file[-5] != 'd':
#                 print(f'Folder: {folder}') 
#                 print(f'File: {file}')
#                 print('')

#                 plot = pd.read_csv(f'{path_split}/{folder}/{file}')

#                 # data = sc.loadmat(f'cutting_tests_processed/{folder}/{file[0:9]}.mat')
#                 # lim = pd.DataFrame.from_dict(data['tsDS'][:,:])
#                 # lim.rename({0: 't', 1: 'y'}, axis=1, inplace=True)
#                 # limit = np.fft.fft(lim['y']).max()

#                 fft = np.fft.fft(plot['y'])
#                 fftfreq = np.fft.fftfreq(len(plot['t']), plot['t'][1] - plot['t'][0])

#                 # ploting
#                 fig, ax = plt.subplots(1,2, figsize = (10,4))
#                 # plt.ylim(0,limit)
#                 ax[0].plot(plot['t'], plot['y'])
#                 ax[1].plot(fftfreq[10:len(fft)//2], np.abs(fft[10:len(fft)//2])/len(fft), c = 'k')
#                 fig.suptitle(file)
#                 plt.show()

#                 # classifing 
#                 print("c = has chatter \n s = no chatter \n \"stop\" \n \"pass\"")
#                 presence = input('')
#                 if presence == 'stop':
#                     break
#                 if presence == 'pass':
#                     continue

#                 # saving files
#                 shutil.copyfile(f'{path_split}/{folder}/{file}', f'{path_corrected}/{folder}/{presence}{file[1:]}')
#                 shutil.copyfile(f'{path_split}/{folder}/{file[:-4]}_noise_added.csv', f'{path_corrected}/{folder}/{presence}{file[1:-4]}_noise_added.csv')

#                 clear_output()


In [14]:
# main_()

# Creating a new data frame with math features 

Defining the functions used in the creation of the dataframe as to keep the formulas more organized and easy to read. 

In [15]:
def thd(data_): # true harmonic distortion
    sq_sum = 0.0
    for r in range(len(data_)):
        sq_sum = sq_sum + data_[r]**2
    sq_harmonics = sq_sum - max(data_)**2
    return 100*sq_harmonics**0.5 / max(data_)

def rms(data_):
    return np.sqrt(np.mean(data_**2))

def amp(data_):
    return max(data_) - min(data_)

def SRA(data_):
    return np.mean(np.sqrt(np.abs(data_)))**2

def crest(data_):
    return max(np.abs(data_))/np.sqrt(np.mean(data_**2))

def impulse(data_):
    return max(np.abs(data_))/np.mean(abs(data_))

def margin(data_):
    return max(abs(data_))/np.mean(np.sqrt(np.abs(data_)))**2

def freq_max(data_):
    data_ = data_[100:]
    # sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]
    peak_freq = data_['fftfreq'][data_['fft'] == data_['fft'].max()].values[0]
#    print(peak_freq)
    return peak_freq

    
def freq_sec_highest(data_):
    # sorted(np.array(scipy.fftpack.fft(file['fftfreq'].values)))[-3]
    data_ = data_[100:]
    data_sorted = data_.sort_values(by = ['fft'], ascending = False)
    return data_sorted['fftfreq'].to_numpy()[1]

def fft_mean(data_):
    ffts_array = np.array([np.abs(complex(i)) for i in data_['fft'].to_numpy()])
    return ffts_array.mean()

# def freq_pico(data_):
#     # sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]
#     return data_['fftfreq'].max()

#         cols['freq_max'].append(scipy.fftpack.fft(file['y'].values).max()) # https://gist.github.com/endolith/255291
#         cols['freq_sec_highest'].append(sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]) 
#         cols['freq_mean'].append(scipy.fftpack.fft(file['y'].values).mean()) 


In [18]:
''' Creating the data frame of math equations outputs.
'''
cols = {'Experiment': [],
        'chatter_presence': [],
        'med': [],
        'std': [],
        'rms': [],
        'curtose': [],
        'distortion': [],
        'amplitude': [],
        'sra': [],
        'crest': [],
        'impulse': [],
        'margin': [],
        'max': [],
        'freq_max': [],
        'freq_sec_highest': [],
        'fft_mean': []}

path = "corrected"
list_of_folders = listdir(path)
for folder in list_of_folders:
    list_of_files = listdir(f'{path}/{folder}')
    for file in list_of_files:
        cols['Experiment'].append(f'{file[:-4]}')
        cols['chatter_presence'].append(f'{file[:1]}')
        
        # print(file)
        file = pd.read_csv(f'{path}/{folder}/{file}', index_col=0)
                
        cols['med'].append(file['y'].mean())
        cols['std'].append(file['y'].std())
        cols['rms'].append(rms(file['y']))
        
        cols['curtose'].append(scipy.stats.kurtosis(file['y']))
        cols['distortion'].append(thd(file['y']))
        cols['amplitude'].append(amp(file['y']))
        cols['sra'].append(SRA(file['y']))
        
        cols['crest'].append(crest(file['y']))
        cols['impulse'].append(impulse(file['y']))
        cols['margin'].append(margin(file['y']))
        cols['max'].append(file['y'].max())
        
        cols['freq_max'].append(freq_max(file)) # https://gist.github.com/endolith/255291
        cols['freq_sec_highest'].append(freq_sec_highest(file)) 
        cols['fft_mean'].append(fft_mean(file))

        clear_output()

dataframe = pd.DataFrame(cols)

In [19]:
dataframe

,Experiment,chatter_presence,med,std,rms,curtose,distortion,amplitude,sra,crest,impulse,margin,max,freq_max,freq_sec_highest,fft_mean
0,s_320_045_split_1,s,0.005482,0.002285,0.005940,-0.684501,13046.608858,0.019401,0.005217,2.709846,2.934679,3.084994,0.016095,3075.253849,3314.593371,0.617542
1,s_320_015_split_0,s,0.005475,0.002081,0.005857,-0.254890,7052.357014,0.041907,0.005266,4.755544,5.084255,5.289695,0.027854,119.866400,696.665119,0.406478
2,s_320_020_split_3,s,0.005492,0.001971,0.005835,-1.170823,8819.340916,0.010459,0.005303,1.792693,1.904703,1.972551,0.010460,-119.598804,-1207.787922,0.159834
3,c_770_005_split_3,c,0.005414,0.040638,0.040996,0.898666,4733.248435,0.416909,0.026842,5.159852,6.625808,7.880688,0.205375,-945.862463,-845.862907,6.028744
4,s_320_005_split_1_noise_added,s,0.005498,0.002290,0.005955,-0.221540,8084.124352,0.022319,0.005245,2.765787,2.989542,3.140532,0.016471,3762.600000,-3762.600000,0.239377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,c_570_012_split_1_noise_added,c,0.005464,0.003003,0.006235,0.603812,7292.776694,0.027757,0.005170,3.065853,3.423945,3.697408,0.019116,1049.000000,-1049.000000,0.294652
624,c_770_005_split_3_noise_added,c,0.005426,0.008870,0.010397,1.034704,1766.022171,0.087902,0.007025,4.583835,5.757399,6.784075,0.040242,1175.592063,-1175.592063,0.315492
625,c_570_012_split_2_noise_added,c,0.005484,0.002065,0.005860,-0.212874,9666.131901,0.016478,0.005264,2.313178,2.470556,2.575268,0.013556,-19.400000,-1067.600000,0.239460
626,c_570_012_split_1,c,0.005461,0.002938,0.006201,0.663894,7286.361721,0.026717,0.005174,3.068551,3.415217,3.677729,0.019029,3492.732536,3601.931990,0.439309


In [ ]:
dataframe.to_csv('math_outputs.csv')

# create df for clustering

In [20]:
""" 
    THIS ONE USES THE UNSPLIT (WITHOUT NOISE)
"""
# ''' Creating the data frame of math equations outputs.
# '''
# clust = {'Experiment': [],
#         'chatter_presence': [],
#         'med': [],
#         'std': [],
#         'rms': [],
#         'curtose': [],
#         'distortion': [],
#         'amplitude': [],
#         'sra': [],
#         'crest': [],
#         'impulse': [],
#         'margin': [],
#         'max': [],
#         'freq_max': [],
#         'freq_sec_highest': [],
#         'fft_mean': []}

# path = "cutting_tests_processed"
# list_of_folders = listdir(path)
# for folder in list_of_folders:
#     list_of_files = listdir(f'{path}/{folder}')
#     for file in list_of_files:
#         clust['Experiment'].append(f'{file[:-4]}')
#         clust['chatter_presence'].append(f'{file[:1]}')
        
#         # print(file)
#         data = sc.loadmat(f'{path}/{folder}/{file}')
#         file = pd.DataFrame.from_dict(data['tsDS'][:,:])
#         file.rename({0: 't', 1: 'y'}, axis=1, inplace=True)
        
#         freq = np.fft.fftfreq(len(file['t']), file['t'][1] - file['t'][0])
#         yf = np.fft.fft(file['y'])
#         file['fft'] = yf
#         file['fftfreq'] = freq # omega
                
#         clust['med'].append(file['y'].mean())
#         clust['std'].append(file['y'].std())
#         clust['rms'].append(rms(file['y']))
        
#         clust['curtose'].append(scipy.stats.kurtosis(file['y']))
#         clust['distortion'].append(thd(file['y']))
#         clust['amplitude'].append(amp(file['y']))
#         clust['sra'].append(SRA(file['y']))
        
#         clust['crest'].append(crest(file['y']))
#         clust['impulse'].append(impulse(file['y']))
#         clust['margin'].append(margin(file['y']))
#         clust['max'].append(file['y'].max())
        
#         clust['freq_max'].append(freq_max(file)) # https://gist.github.com/endolith/255291
#         clust['freq_sec_highest'].append(freq_sec_highest(file)) 
#         clust['fft_mean'].append(fft_mean(file))

#         clear_output()

# dataframe_cluster = pd.DataFrame(clust)

In [27]:
''' Creating the data frame of math equations outputs.
'''
clust = {'Experiment': [],
        'chatter_presence': [],
        'med': [],
        'std': [],
        'rms': [],
        'curtose': [],
        'distortion': [],
        'amplitude': [],
        'sra': [],
        'crest': [],
        'impulse': [],
        'margin': [],
        'max': [],
        'freq_max': [],
        'freq_sec_highest': [],
        'fft_mean': []}

path = "split_cutting_tests_processed"
list_of_folders = listdir(path)
for folder in list_of_folders:
    list_of_files = listdir(f'{path}/{folder}')
    for file in list_of_files:
        if file[-5] != 'd':
            clust['Experiment'].append(f'{file[:-4]}')
            clust['chatter_presence'].append(f'{file[:1]}')
            
            file = pd.read_csv(f'{path}/{folder}/{file}', index_col=0)
                    
            clust['med'].append(file['y'].mean())
            clust['std'].append(file['y'].std())
            clust['rms'].append(rms(file['y']))
            
            clust['curtose'].append(scipy.stats.kurtosis(file['y']))
            clust['distortion'].append(thd(file['y']))
            clust['amplitude'].append(amp(file['y']))
            clust['sra'].append(SRA(file['y']))
            
            clust['crest'].append(crest(file['y']))
            clust['impulse'].append(impulse(file['y']))
            clust['margin'].append(margin(file['y']))
            clust['max'].append(file['y'].max())
            
            clust['freq_max'].append(freq_max(file)) # https://gist.github.com/endolith/255291
            clust['freq_sec_highest'].append(freq_sec_highest(file)) 
            clust['fft_mean'].append(fft_mean(file))

            clear_output()

dataframe_cluster = pd.DataFrame(clust)

In [28]:
dataframe_cluster.head()

,Experiment,chatter_presence,med,std,rms,curtose,distortion,amplitude,sra,crest,impulse,margin,max,freq_max,freq_sec_highest,fft_mean
0,s_320_045_split_1,s,0.005482,0.002285,0.005940,-0.684501,13046.608858,0.019401,0.005217,2.709846,2.934679,3.084994,0.016095,3075.253849,3314.593371,0.617542
1,s_320_015_split_0,s,0.005475,0.002081,0.005857,-0.254890,7052.357014,0.041907,0.005266,4.755544,5.084255,5.289695,0.027854,119.866400,696.665119,0.406478
2,s_320_020_split_3,s,0.005492,0.001971,0.005835,-1.170823,8819.340916,0.010459,0.005303,1.792693,1.904703,1.972551,0.010460,-119.598804,-1207.787922,0.159834
3,c_770_005_split_3,c,0.005414,0.040638,0.040996,0.898666,4733.248435,0.416909,0.026842,5.159852,6.625808,7.880688,0.205375,-945.862463,-845.862907,6.028744
4,c_770_002_split_0,c,0.005469,0.014140,0.015160,3.312443,1899.488742,0.160458,0.008789,5.493718,7.627771,9.475498,0.077174,933.575105,959.974401,1.305385


In [29]:
dataframe_cluster.to_csv("for_clustering_math_outputs.csv")